### Développement d’un pipeline de calcul du TMB

#### Projet 4BiM, 2021

#### Auteurs : Marie Casimir, Loup Petitjean et Nicolas Mendiboure
#### Encadrantes Innate-Pharma : Sabrina Carpentier et Luciana Bastista
#### Encadrante INSA : Maïwenn Pineau

*Pour install PyVCF : pip install pyvcf*

In [ ]:
import os
import vcf
import pandas as pds

In [ ]:
# Pour charger un fichier vcf :

def read_vcf(f):
    
    reader = vcf.Reader(open(f))
    df = pds.DataFrame([vars(r) for r in reader])
    out = df.merge(pds.DataFrame(df.INFO.tolist()),
                   left_index=True, right_index=True)
    return out

In [ ]:
my_file = "./datas.vcf"
read_vcf(my_file).head()